<a href="https://colab.research.google.com/github/karankulshrestha/ai-notebooks/blob/main/SimpleGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameter
batch_size = 16 # how many sentences at once want to process.
block_size = 32 # number of words our each sentence contains.
max_iters = 5000 # number of times, we train the model
eval_interval = 100
learning_rate = 1e-3 # learning rate is used to scale and shift the parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 # number of times we test our model
n_embed = 64 # number of features our each word of the sentence have, in this case 64 features.
n_head = 4 # number of parallel self-attentions scores are calculated for each sentence.
n_layer = 4 # number of layer of transformer a sentence pass through for deep understanding, like Input → Layer 1 → Layer 2 → Layer 3 → Layer 4 → Output
dropout = 0.0 # how much neurons we forget during training, for preventing overfitting.

#----------------

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create bi-directional mapping from char to int
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }

# ecoder and decode for string
encode = lambda s: [stoi[ch] for ch in s] # take input of string and output list of integers.
decode = lambda l: ''.join([itos[i] for i in l]) # take input of list of integers and output string.

# Train and Test Data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
  # generate the batch of input x and target y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y


# evaluation of the model after certain steps of training
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y = get_batch(split)
        logits, loss = model(X, Y)
        losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


class Head(nn.Module):
  """One head of self-attention"""
  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('trill', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    # tokens attentions scores
    wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
    wei = wei.masked_fill(self.trill[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    # tokens weight aggregation
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)

    return out



class MultiHeadAttention(nn.Module):
  """ Multiple-Head of self attention in parallel to capture more relations between tokens """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out



class FeedForward(nn.Module):

  """ A simple feed forward neural network for capturing more details"""

  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4 * n_embed),
        nn.ReLU(),
        nn.Linear(4 * n_embed, n_embed),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)



class Block(nn.Module):
  """ Transformer Block """

  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size) # calculate attention scores for each sequence and concatenate the n_head different scores in last dimension.
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x




class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed) # final layer norm
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape # number of batches B and each batch contains a sequence of T tokens.

    # idx and targets are both (B,T) tensor of integers
    token_emb = self.token_embedding_table(idx) # (B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = token_emb + pos_emb # (B,T,C)
    x = self.blocks(x) # (B,T,C)
    x = self.ln_f(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
      # this is the context window size, last context to remember
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last token in every sequence across batches
      logits = logits[:, -1, :] # becomes (B, C) logits = [[vocab_scores_for_last_token_seq1], [vocab_scores_for_last_token_seq2], ...]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) [[index1_token_1_seq1], [index2_token_2_seq2], [index3_token_3_seq3],...]
      # append the next_idx to the previous idx
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


model = BigramLanguageModel()
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer -> make previous gradients 0 -> compute gradients of loss w.r.t to model.parameters() [weights] -> backpropagate (update the weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# training
for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train') # basically xb (B, T) -> input and yb -> output (B, T) [each token shifted by 1 offset corresponding to token in xb]

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


--2025-06-23 14:47:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2025-06-23 14:47:52 (129 MB/s) - ‘input.txt.4’ saved [1115394/1115394]

0.209729 M parameters
step 0: train loss 4.4111, val loss 4.4018
step 100: train loss 2.6551, val loss 2.6645
step 200: train loss 2.5084, val loss 2.5036
step 300: train loss 2.4155, val loss 2.4296
step 400: train loss 2.3477, val loss 2.3539
step 500: train loss 2.2966, val loss 2.3118
step 600: train loss 2.2411, val loss 2.2500
step 700: train loss 2.2019, val loss 2.2157
step 800: train lo